In [45]:
#Spell Checker using noisy channel correction

#Logic to be constructed!
#construct confusion matrix using the file spell_error.txt all sub, ins, del, trans -- This is the model
#read test data file.
#for the first word generate the candidate generation words using one edit distance.
#While generating the candidate pairs, send the possible candidate and also the edit type: sub, ins, del, trans
#for all the candidate pairs loop
# if the edit type is del, look up for del matrix.
# In the del matrix search for del[wi-i,wi]/count[wi-iwi], count is the count from the count mapped against w correct candidate
#Calculate individal probabilties and save the result. Here p(w) is calculated against the total count of words
# for all the candidates look for max probablites and that will be p(w/x). This has to be the results
# Save it against the record.


In [46]:
%pylab inline
import re
import math
import string
from collections import Counter
from __future__ import division
import csv
import pandas as pd
import re 
from collections import Counter


Populating the interactive namespace from numpy and matplotlib


/Users/joby/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [47]:
#File_Corpus = 'test_data/count_1w.txt'

File_Corpus = 'test_data/big.txt'
File_Spell_Errors='test_data/spell-errors.txt'

Test_Data = 'test_data/test.csv'

text = open(File_Spell_Errors).read()
lines = text.split("\n")
dictionary = {}
for line in lines:
    key = line.strip().split(": ")[0]
    value = line.strip().split(": ")[1]
    list1 = value.split(", ")
    dictionary[key] = list1

def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open(File_Corpus).read()))

def read_csv_file(filename):
    error_words = []
 
    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        next(csvReader)
        for row in csvReader:
            error_words.append(row[1])
 
    return error_words
    
    
def test_corpus(filename):
    text = open(filename).read()
    return dict((x.split(' ')[0],int(x.split(' ')[1])) for x in text.split('\n'))

#WORDS = test_corpus(File_Corpus)

test_words = read_csv_file(Test_Data)

def write_to_file(wordlist):
    df = pd.DataFrame(wordlist)
    df.to_csv("Output/test_submit_main.csv")

    

In [48]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(WORDS)


In [49]:
def correction(word): 
    "Most probable spelling correction for word."    
    #print(candidates(word))
    list_correct_words = []
    candidate_words = candidates(word)
    for rightWord in candidate_words:
        if rightWord in dictionary:
            if word in dictionary.get(rightWord):
                list_correct_words.append(rightWord)
    if list_correct_words:
        return max(list_correct_words, key=P)
    return max(candidate_words, key = P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS.keys())

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [50]:
#P('imidatly')*math.pow(10,10)
print(correction('nessisitates'))


nessisitates


In [51]:
correct_list=[]
for w in test_words:
    correct_list.append(correction(w))

#for w in correct_list:
 #   print(w)

In [52]:
#Saving to file

write_to_file(correct_list)